Implementing a Custom Join 
==============

# Setup

In [0]:
data_version = "1y" # "3m", "6m", "1y", "" -> blank is full

In [0]:
# Imports

from pyspark.sql.functions import col, regexp_replace, split, trim, to_timestamp, date_format, broadcast
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pandas as pd

##### Spark settings for efficient execution

In [0]:
spark.conf.get("spark.sql.session.timeZone")
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [0]:
# Enable Adaptive Query Execution (AQE) and optimizations for joins
spark.conf.set("spark.sql.adaptive.enabled", "true")                  # Adaptive Query Execution (auto optimizes joins/shuffles)
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")         # Automatically handles skewed joins
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")  # Merge small partitions post-shuffle
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 50 * 1024 * 1024) # Optional: increase broadcast threshold to 50MB


Reading back to see the spark settings

In [0]:
print("Spark version:", spark.version)
print("AQE:", spark.conf.get("spark.sql.adaptive.enabled"))
print("Skew join:", spark.conf.get("spark.sql.adaptive.skewJoin.enabled"))
print("Coalesce:", spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled"))
print("AutoBroadcastJoinThreshold:", spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))


In [0]:
# Paths & Defines

data_BASE_DIR = "dbfs:/mnt/mids-w261/"
display(dbutils.fs.ls(f"{data_BASE_DIR}")) 

# Section Folder
section = "4"
number = "2"
section_DIR = f"dbfs:/mnt/mids-w261/student-groups/Group_{section}_{number}"

# Subdirectories for organization
processed_DIR = f"{section_DIR}/processed"
checkpoints_DIR = f"{section_DIR}/checkpoints"
intermediate_DIR = f"{section_DIR}/intermediate"

# Output filenames (using variables for maintainability)
FLIGHTS_WITH_STATION = f"flights_with_station_{data_version}"
FLIGHTS_WEATHER_JOINED = f"flights_weather_joined_{data_version}"
FLIGHTS_WITH_AIRPORTS = f"flights_with_airports_{data_version}"

# Full paths for outputs
flights_with_station_path = f"{intermediate_DIR}/{FLIGHTS_WITH_STATION}"
flights_weather_joined_path = f"{processed_DIR}/{FLIGHTS_WEATHER_JOINED}"
flights_with_airports_path = f"{intermediate_DIR}/{FLIGHTS_WITH_AIRPORTS}"

# Check if section_DIR exists, print contents or create it
try:
    print(f"✓ Section directory exists: {section_DIR}")
    print("\nContents:")
    contents = dbutils.fs.ls(section_DIR)
    for item in contents:
        print(f"  - {item.name} ({'DIR' if item.isDir() else 'FILE'}) - {item.size} bytes")
    print(f"\nTotal items: {len(contents)}")
except Exception as e:
    print(f"✗ Section directory does not exist: {section_DIR}")
    print("Creating directory structure...")
    dbutils.fs.mkdirs(section_DIR)
    print(f"✓ Base directory created: {section_DIR}")

# Create subdirectories
print("\nCreating/verifying subdirectories...")
for subdir_name, subdir_path in [
    ("processed", processed_DIR),
    ("checkpoints", checkpoints_DIR),
    ("intermediate", intermediate_DIR)
]:
    try:
        dbutils.fs.mkdirs(subdir_path)
        print(f"✓ {subdir_name}: {subdir_path}")
    except Exception as e:
        print(f"✗ Error creating {subdir_name}: {e}")

# Set checkpoint directory for Spark
spark.sparkContext.setCheckpointDir(checkpoints_DIR)

print("\n" + "="*60)
print("Directory structure ready!")
print("="*60)
print(f"\nKey paths:")
print(f"  Intermediate: {intermediate_DIR}")
print(f"  Processed: {processed_DIR}")
print(f"  Checkpoints: {checkpoints_DIR}")
print(f"\nOutput files:")
print(f"  Flights+Station: {flights_with_station_path}")
print(f"  Flights+Weather: {flights_weather_joined_path}")

# Load Data

#### Provided Datasets

In [0]:
dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project_2022/")

In [0]:
# Airline Data    
if data_version == "":
    df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/") # full dataset
else:
    df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_{data_version}/") 

# Stations data      
df_stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/")

# Weather data
if data_version == "":
    df_weather = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/") # full dataset
else:
    df_weather = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_{data_version}/")


In [0]:
df_flights.printSchema()

In [0]:
df_weather.printSchema()

In [0]:
df_stations.printSchema()

#### Airport Codes - Additional 

This adds missing data in DF Flights, like Latitude & Longitude

In [0]:
# Download the CSV file to DBFS
import requests

url = "https://datahub.io/core/airport-codes/r/airport-codes.csv"
local_path = "/dbfs/tmp/airport-codes.csv"

response = requests.get(url)
with open(local_path, "wb") as f:
    f.write(response.content)


# Use the corresponding DBFS path for Spark
dbfs_path = "dbfs:/tmp/airport-codes.csv"

# Copy to DBFS
dbutils.fs.cp("file:" + local_path, dbfs_path)

# Read the CSV file from DBFS
df_airport_codes = spark.read.option("header", "true").csv("/tmp/airport-codes.csv")

display(df_airport_codes)

In [0]:
df_airport_codes.printSchema()

In [0]:
df_airport_codes.describe()

In [0]:
display(df_airport_codes.limit(5))

#### Airport - Timezone string - Additional Dataset

Source: https://github.com/opentraveldata/opentraveldata/blob/master/opentraveldata/optd_por_public.csv 


In [0]:
url = "https://raw.githubusercontent.com/opentraveldata/opentraveldata/master/opentraveldata/optd_por_public.csv"

local_path = "/dbfs/tmp/airport-timezones.csv"

with open(local_path, "wb") as f:
    f.write(requests.get(url).content)

# Use the corresponding DBFS path for Spark
dbfs_path = "dbfs:/tmp/airport-timezones.csv"

# Copy to DBFS
dbutils.fs.cp("file:" + local_path, dbfs_path)

df_airport_timezones = (
    spark.read
        .option("header", True)
        .option("delimiter", "^")
        .option("inferSchema", True)
        .csv(dbfs_path)
)

cols_to_keep = [
    "iata_code",
    "icao_code",
    "faa_code",
    "timezone",
    "latitude",
    "longitude"
]

df_airport_timezones = df_airport_timezones.select(cols_to_keep)

display(df_airport_timezones)

In [0]:
df_airport_timezones.summary()

In [0]:
# Join airport_codes table with airport_timezones 

# Perform a left join on both iata_code and icao_code
df_airport_joined = (
    df_airport_codes.alias("a")
    .join(
        df_airport_timezones.alias("b"),
        (
            (F.col("a.iata_code") == F.col("b.iata_code")) |
            (F.col("a.icao_code") == F.col("b.icao_code"))
        ),
        how="left"
    )
)

# Display result
display(df_airport_joined) 

In [0]:
print("Row count before join:", df_airport_codes.count())
print("Row count after join:", df_airport_joined.count())

In [0]:
df_airport_joined_clean = df_airport_joined.select(
    "ident", "type", "name", "elevation_ft",
    "continent", "iso_country", "iso_region", "municipality",
    "a.icao_code", "a.iata_code", "gps_code", "local_code",
    "coordinates",
    F.col("b.latitude").alias("latitude"),
    F.col("b.longitude").alias("longitude"),
    F.col("b.timezone").alias("timezone"),
    F.col("b.faa_code").alias("faa_code_otd")
)

display(df_airport_joined_clean)


In [0]:

df_airport_joined = df_airport_joined_clean

# Define window partitioned by your key columns
w = Window.partitionBy("iata_code", "icao_code").orderBy(
    F.when(F.col("latitude").isNotNull(), 1).otherwise(2),  # prefer non-null lat
    F.when(F.col("timezone").isNotNull(), 1).otherwise(2)   # then prefer non-null tz
)

# Add row number within each partition and keep the first
df_airport_dedup = (
    df_airport_joined
    .withColumn("row_num", F.row_number().over(w))
    .filter(F.col("row_num") == 1)
    .drop("row_num")
)

print("Row count before de-dup:", df_airport_joined.count())
print("Row count after de-dup:", df_airport_dedup.count()) 

In [0]:
# Add timestamp column

df_flights = df_flights.withColumn(
    "fl_date_timestamp",
    to_timestamp(col("fl_date"), "yyyy-MM-dd")
)

# Parse full datetime string (keep hours/minutes)
df_weather = df_weather.withColumn(
    "date_timestamp",
    F.to_timestamp(F.col("date"), "yyyy-MM-dd'T'HH:mm:ss")
)

In [0]:
display(df_airport_dedup.select("timezone").groupBy("timezone").count().orderBy("count", ascending=False))

In [0]:
df_airports = df_airport_dedup

In [0]:
sizes = {
    "Stations": df_stations.count() * len(df_stations.columns),
    "Weather": df_weather.count() * len(df_weather.columns),
    "Flights": df_flights.count() * len(df_flights.columns),
    "Airports": df_airports.count() * len(df_airports.columns)
}

print("\n DataFrame Size Summary\n" + "-" * 30)
for name, size in sizes.items():
    print(f"{name:<10}: {size:,}")

In [0]:
display(
    df_airports.filter(F.col("timezone").isNull())
    .select("iso_region")
    .distinct()
)

#### Pre-processing

Convert column names into lower case

In [0]:

# Convert all column names to lowercase
df_flights = df_flights.toDF(*[c.lower() for c in df_flights.columns])
df_weather = df_weather.toDF(*[c.lower() for c in df_weather.columns])
df_stations = df_stations.toDF(*[c.lower() for c in df_stations.columns])
df_airports = df_airports.toDF(*[c.lower() for c in df_airports.columns])


##### Split coordinates column into latitude and longitude

In [0]:
# create latitude and longitude colums in Airport Codes
df_airports = df_airports.withColumn(
    "coordinates",
    regexp_replace("coordinates", "[()]", "")
)

df_airports = df_airports.withColumn(
    "lat_lon",
    split("coordinates", ",")
)

df_airports = df_airports.withColumn(
    "latitude",
    trim(df_airports["lat_lon"].getItem(0))
).withColumn(
    "longitude",
    trim(df_airports["lat_lon"].getItem(1))
).drop("lat_lon")

display(df_airports)

In [0]:
display(df_stations.limit(5))

##### create neighbor_iata column

Since Stations data's contains 4 digit ICAO codes, we need to  convert it to 3 digit IATA codes for US airports.  

Example: 
ICAO (4-letter): e.g., KJFK, KLAX, EGLL
→ This is what neighbor_call contains.

IATA (3-letter): e.g., JFK, LAX, LHR
→ This is what flight data uses.



In [0]:

df_stations = df_stations.withColumn(
    "neighbor_iata",
    F.expr("substring(neighbor_call, 2, 3)")  # remove leading 'K' for US airports
) 

In [0]:
display(df_stations.select("neighbor_call","neighbor_iata").limit(5))

##### create scheduled departure time  (local)

In [0]:
from pyspark.sql.functions import col, lpad, concat, to_timestamp, expr

# Combine fl_date_timestamp and crs_dep_time to create sched_depart_date_time
df_flights = df_flights.withColumn(
    "sched_depart_date_time",
    to_timestamp(
        concat(
            col("fl_date"),
            lpad(col("crs_dep_time"), 4, "0")
        ),
        "yyyy-MM-ddHHmm"
    )
)


In [0]:
display(df_airport_codes.limit(5))

# Implement Joins

#### Step 1: Join Flights data with Airports

This join is done using IATA code of the airport. We do this join to get data like latitude/longitude of the airport - both origin & destination. 

In [0]:
# Verify airports table is small enough to broadcast
print(f"Airports count: {df_airports.count()}")
print(f"Airports size: {df_airports.rdd.map(lambda x: len(str(x))).sum() / 1024 / 1024} MB")


In [0]:

# Prepare airports dataframe - select only needed columns
df_airports_clean = df_airports.select(
    col("iata_code"),
    col("name"),
    col("latitude").cast("double").alias("latitude"),
    col("longitude").cast("double").alias("longitude"),
    col("iso_country"),
    col("timezone")
).filter(
    col("iata_code").isNotNull() &
    col("latitude").isNotNull() &
    col("longitude").isNotNull()
).cache()

print(f"Airports count: {df_airports_clean.count()}")

# Broadcast airports (it's small enough)
airports_broadcast = broadcast(df_airports_clean)

# Perform the join
df_flights_with_airports = (
    df_flights.alias("f")
    .join(
        airports_broadcast.alias("ao"),
        col("f.origin") == col("ao.iata_code"),
        "left"
    )
    .join(
        airports_broadcast.alias("ad"),
        col("f.dest") == col("ad.iata_code"),
        "left"
    )
    .select(
        col("f.*"),
        # Origin airport info
        col("ao.name").alias("origin_airport_name"),
        col("ao.latitude").alias("origin_latitude"),
        col("ao.longitude").alias("origin_longitude"),
        col("ao.iso_country").alias("origin_country"),
        col("ao.timezone").alias("origin_timezone"),
        # Destination airport info
        col("ad.name").alias("destination_airport_name"),
        col("ad.latitude").alias("destination_latitude"),
        col("ad.longitude").alias("destination_longitude"),
        col("ad.iso_country").alias("destination_country"),
        col("ad.timezone").alias("destination_timezone")
    )
)


Get all timestamps in UTC using timezone info

In [0]:
df_flights_with_airports = df_flights_with_airports.withColumn(
    "sched_depart_date_time_UTC",
    F.to_utc_timestamp("sched_depart_date_time", F.col("origin_timezone"))
)

# Two hours prior to scheduled departure in UTC
df_flights_with_airports = df_flights_with_airports.withColumn(
    "two_hours_prior_depart_UTC",
    expr("sched_depart_date_time_UTC - INTERVAL 2 HOURS")
)

# Four hours prior to scheduled departure in UTC
df_flights_with_airports = df_flights_with_airports.withColumn(
    "four_hours_prior_depart_UTC",
    expr("sched_depart_date_time_UTC - INTERVAL 4 HOURS")
)

display(df_flights_with_airports.select(
    "fl_date", "crs_dep_time", "sched_depart_date_time",
    "sched_depart_date_time_UTC", "two_hours_prior_depart_UTC", "four_hours_prior_depart_UTC"
).limit(5))

In [0]:
display(df_weather.limit(5))

In [0]:

# Verify join quality
print(f"\nOriginal flights: {df_flights.count():,}")
print(f"After join: {df_flights_with_airports.count():,}")
print(f"Missing origin coords: {df_flights_with_airports.filter(col('origin_latitude').isNull()).count():,}")

print("\n✓ Join 1 Complete")

##### Checkpoint 1: Store after first join 

In [0]:
# df_flights_combined 

# df_flights_with_station_notnull = spark.read.parquet(f"{section_DIR}/df_flights_with_station_{data_version}")

#### Step 2: Join Flights+Airport with Stations 

Here we try two different approaches:
 - Using neighbor_call/neighbor_iata 
 - Using latitude & longitude


In [0]:
display(df_stations.limit(5))

So that shows all of them match! 

##### Join type 2: get stationID from stations table using lat/long

Join using Origin airport code and neighbor_iata column of stations data and distance to neighbor < 250km. 

Then check for nulls.

##### Join type 2: get stationID from stations table using lat/long



Doing a cross-join with the whole flights data & station data will create too many combinations. It's better we only get the join done with distinct airports and later combine it with original flights data.

TODO: Try Haversine approach

In [0]:

# Step 1: Get distinct origin airports
df_distinct_origins = (
    df_flights_with_airports
    .select("origin", "origin_latitude", "origin_longitude")
    .distinct()
    .filter(
        col("origin_latitude").isNotNull() & 
        col("origin_longitude").isNotNull()
    )
    .repartition(200, "origin")
    .cache()
)

print(f"Distinct origins: {df_distinct_origins.count()}")

# Step 2: Prepare stations dataframe
df_stations_clean = df_stations.select(
    col("station_id"),
    col("lat").cast("double"),
    col("lon").cast("double")
).filter(
    col("station_id").isNotNull() &
    col("lat").isNotNull() &
    col("lon").isNotNull()
)

stations_broadcast = broadcast(df_stations_clean)

# Step 3: Perform spatial join with bounding box filter
df_candidates = (
    df_distinct_origins.alias("a")
    .join(
        stations_broadcast.alias("s"),
        (col("s.lat").between(col("a.origin_latitude") - 0.5, col("a.origin_latitude") + 0.5)) &
        (col("s.lon").between(col("a.origin_longitude") - 0.5, col("a.origin_longitude") + 0.5)),
        "inner"
    )
    .withColumn(
        "distance_km",
        F.expr("""
            6371 * acos(
                least(1.0,
                    cos(radians(a.origin_latitude)) * cos(radians(s.lat)) *
                    cos(radians(s.lon) - radians(a.origin_longitude)) +
                    sin(radians(a.origin_latitude)) * sin(radians(s.lat))
                )
            )
        """)
    )
    .filter(col("distance_km") < 50)
)

# Step 4: Get nearest station per airport
window_spec = Window.partitionBy("a.origin").orderBy("distance_km")

df_nearest_stations = (
    df_candidates
    .withColumn("rank", F.row_number().over(window_spec))
    .filter(col("rank") == 1)
    .select(
        col("a.origin").alias("origin"),
        col("s.station_id").alias("origin_station_id"),
        col("distance_km").alias("station_distance_km")
    )
)

print(f"Airports with stations: {df_nearest_stations.count()}")


In [0]:

# Step 5: Join back to main flights dataframe
df_flights_with_station = (
    df_flights_with_airports
    .join(df_nearest_stations, on="origin", how="left")
)

# Check quality
null_count = df_flights_with_station.filter(col("origin_station_id").isNull()).count()
print(f"Flights without station: {null_count:,}")

# Filter out flights without stations
df_flights_with_station_clean = df_flights_with_station.filter(
    col("origin_station_id").isNotNull()
)

print(f"Final flights with stations: {df_flights_with_station_clean.count():,}")
print("\n✓ Join 2 Complete")

In [0]:


# Unpersist cached data
df_distinct_origins.unpersist()

display(df_flights_with_station_clean.limit(5))




Inspect the columns where origin_station_id is null

In [0]:
# display(df_flights_with_station.filter(F.col("origin_station_id").isNull()))

In [0]:
# display(df_flights_with_station.filter(F.col("origin_station_id").isNull()).select('origin').distinct())

Most of the ones which are null are:

- PSE — Mercedita Airport, Ponce, Puerto Rico 🇵🇷
- GUM — Antonio B. Won Pat International Airport, Guam 🇬🇺
- PPG — Pago Pago International Airport (Tafuna Airport), American Samoa 🇦🇸
- ISN — Sloulin Field International Airport (now replaced by Williston Basin International Airport), Williston, North Dakota, USA 🇺🇸

In [0]:
df_flights_with_station_notnull = df_flights_with_station.filter(F.col("origin_station_id").isNotNull())
# display(df_flights_with_station_notnull.limit(5))


##### Checkpoint 2: Before Final Join



In [0]:
# df_flights_with_station_notnull.write.mode("overwrite").parquet(f"{section_DIR}/df_flights_with_station_{data_version}")

# Save intermediate result
#df_flights_with_station_notnull.write.mode("overwrite").parquet(flights_with_station_path)



In [0]:

# Load intermediate result later
#df_flights_with_station_notnull = spark.read.parquet(flights_with_station_path)


#### Step 3: Join Flights+Airports+Stations with Weather 

##### Run the full join

In [0]:
# Step 1: Add time-based partitioning key (6-hour buckets)
df_flights_keyed = (
    df_flights_with_station_clean
    .withColumn(
        "time_bucket",
        F.expr("floor(unix_timestamp(two_hours_prior_depart_UTC) / 21600)")  # 6-hour buckets
    )
)

df_weather_keyed = (
    df_weather
    .withColumn(
        "time_bucket",
        F.expr("floor(unix_timestamp(date_timestamp) / 21600)")  # 6-hour buckets
    )
    .filter(
        F.col("station").isNotNull() &
        col("date_timestamp").isNotNull()
    )
)

print("Time bucketing complete")

# Step 2: Repartition by station and time bucket
df_flights_partitioned = df_flights_keyed.repartition(400, "origin_station_id", "time_bucket")
df_weather_partitioned = df_weather_keyed.repartition(400, "station", "time_bucket")

print("Partitioning complete")

# Step 3: Join with time bucket AND adjacent buckets (handles midnight crossover)
df_weather_join = (
    df_flights_partitioned.alias("f")
    .join(
        df_weather_partitioned.alias("w"),
        (col("f.origin_station_id") == col("w.station")) &
        (
            (col("f.time_bucket") == col("w.time_bucket")) |
            #(col("f.time_bucket") == col("w.time_bucket") + 1) |
            (col("f.time_bucket") == col("w.time_bucket") - 1)
        ) &
        (col("w.date_timestamp").between(
            col("f.two_hours_prior_depart_UTC") - F.expr("INTERVAL 1 HOURS"),
            col("f.two_hours_prior_depart_UTC")
        )),
        "left"
    )
)

print("Join complete, ranking weather records...")

# Step 4: Get closest weather record
df_weather_ranked = (
    df_weather_join
    .withColumn(
        "time_diff_sec",
        F.abs(
            F.unix_timestamp("f.two_hours_prior_depart_UTC") - 
            F.unix_timestamp("w.date_timestamp")
        )
    )
)

window_closest = Window.partitionBy(
    "f.origin",
    "f.fl_date",
    "f.crs_dep_time",
    "f.op_carrier_fl_num",
    "f.two_hours_prior_depart_UTC"
).orderBy(col("time_diff_sec").asc_nulls_last())

df_final = (
    df_weather_ranked
    .withColumn("weather_rank", F.row_number().over(window_closest))
    .filter(col("weather_rank") == 1)
    .drop("weather_rank", "time_diff_sec", "time_bucket")
)


# CRITICAL: Checkpoint to break lineage
print("\n⚡ Checkpointing result to break lineage...")
spark.sparkContext.setCheckpointDir(checkpoints_DIR)

# - Break lineage immediately after join
print("\nBreaking lineage with checkpoint...")
df_final = df_final.checkpoint(eager=True)

print(f"Final row count: {df_final.count():,}")
print("\n✓ Join 3 Complete (checkpointed)")



In [0]:
print("Verifying midnight crossover handling:")

midnight_flights = (
    df_final
    .filter(
        (F.hour("sched_depart_date_time_utc") == 0) |
        (F.hour("sched_depart_date_time_utc") == 1)
    )
    .select(
        "origin",
        "sched_depart_date_time_utc",
        "two_hours_prior_depart_utc",
        "w.date_timestamp",
        F.to_date("sched_depart_date_time_utc").alias("flight_date"),
        F.to_date("w.date_timestamp").alias("weather_date"),
        F.datediff(
            F.to_date("w.date_timestamp"),
            F.to_date("sched_depart_date_time_utc")
        ).alias("date_diff")
    )
)

print("Sample midnight flights:")
display(midnight_flights.limit(10))

prev_day_weather = midnight_flights.filter(col("date_diff") == -1).count()
print(f"\nFlights with weather from previous day: {prev_day_weather:,}")


In [0]:

print("Final Data Quality Summary:")
df_final.select(
    F.count("*").alias("total_rows"),
    F.count("w.station").alias("has_weather"),
    F.sum(F.when(col("w.station").isNull(), 1).otherwise(0)).alias("missing_weather"),
    F.count("w.hourlyvisibility").alias("has_visibility"),
    F.round(F.count("w.station") / F.count("*") * 100, 2).alias("weather_coverage_pct")
).show()

# COMMAND ----------
display(df_final.limit(10))

In [0]:

# Check what's currently cached
print("Currently Cached DataFrames:")
print("="*60)
for (id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
    print(f"  RDD ID: {id}")

# Unpersist specific dataframes
df_airports_clean.unpersist()
df_distinct_origins.unpersist()

# If you're not sure what's cached, unpersist everything
spark.catalog.clearCache()

print("✓ Cache cleared")

In [0]:
def check_memory_usage():
    """Check current memory usage"""
    
    print("Memory Usage Report")
    print("="*60)
    
    # 1. Check cached data
    try:
        cached_tables = spark.sql("SHOW TABLES").filter(col("isTemporary") == True)
        print(f"\n1. Cached Tables: {cached_tables.count()}")
    except:
        pass
    
    # 2. Check RDD cache
    cached_rdds = spark.sparkContext._jsc.getPersistentRDDs()
    print(f"\n2. Cached RDDs: {len(cached_rdds)}")
    
    # 3. Check broadcast variables
    try:
        broadcast_count = len(spark.sparkContext._jsc.sc().getBroadcastVariables())
        print(f"\n3. Broadcast Variables: {broadcast_count}")
    except:
        print(f"\n3. Broadcast Variables: Unknown")
    
    # 4. Storage memory
    try:
        storage_status = spark.sparkContext._jsc.sc().getExecutorStorageStatus()
        total_memory = sum([s.maxMem() for s in storage_status])
        used_memory = sum([s.memUsed() for s in storage_status])
        
        print(f"\n4. Storage Memory:")
        print(f"   Used: {used_memory / 1024**3:.2f} GB")
        print(f"   Total: {total_memory / 1024**3:.2f} GB")
        print(f"   Usage: {used_memory / total_memory * 100:.1f}%")
    except Exception as e:
        print(f"\n4. Storage Memory: Unable to retrieve")
    
    print("="*60)

# Check before and after cleanup
check_memory_usage()

In [0]:

import gc

# Python garbage collection
gc.collect()

# JVM garbage collection (more aggressive)
spark.sparkContext._jvm.System.gc()

print("✓ Garbage collection complete")

In [0]:
# Check for duplicate columns
print("Checking for duplicate columns...")
print("="*60)

columns = df_final.columns
duplicates = [col for col in set(columns) if columns.count(col) > 1]

if duplicates:
    print(f"❌ Found duplicate columns: {duplicates}")
    print(f"\nAll columns ({len(columns)}):")
    for i, col in enumerate(columns):
        print(f"  {i+1}. {col}")
else:
    print("✓ No duplicates found")

In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Save Final Result

# COMMAND ----------
print("="*60)
print("SAVING FINAL RESULT")
print("="*60)

# Check current partitioning
num_partitions = df_final.rdd.getNumPartitions()
print(f"Current partitions: {num_partitions}")

# Only repartition if necessary
if num_partitions > 500:
    print(f"⚠ Too many partitions, coalescing to 200")
    df_final = df_final.coalesce(200)
elif num_partitions < 10:
    print(f"⚠ Too few partitions, repartitioning to 50")
    df_final = df_final.repartition(50)
else:
    print(f"✓ Partition count looks good")

df_final = df_final.drop("year") 
# Write
df_final.write.mode("overwrite").parquet(flights_weather_joined_path)
print(f"✓ Data saved to: {flights_weather_joined_path}")

# Quick validation (optional - comment out if too slow)
# df_verification = spark.read.parquet(flights_weather_joined_path)
# print(f"Verification count: {df_verification.count():,}")

# Cleanup
df_airports_clean.unpersist()

print("="*60)
print("ALL JOINS COMPLETE!")
print("="*60)

In [0]:

# Quick validation (optional - comment out if too slow)
df_verification = spark.read.parquet(flights_weather_joined_path)
print(f"Verification count: {df_verification.count():,}")

# OTPW Data

In [0]:

#df_otpw = spark.read.format("csv").option("header","true").load("dbfs:/mnt/mids-w261/OTPW_3M_2015.csv")

df_otpw = spark.read.format("parque").load("dbfs:/mnt/mids-w261/OTPW_12M/")

df_otpw = df_otpw.toDF(*[c.lower() for c in df_otpw.columns])


df_otpw_parsed = df_otpw.withColumn(
    "sched_depart_date_time_ts",
    F.to_timestamp("sched_depart_date_time", "yyyy-MM-dd'T'HH:mm:ss")
).withColumn(
    "sched_depart_date_time_utc_ts",
    F.to_timestamp("sched_depart_date_time_utc", "yyyy-MM-dd'T'HH:mm:ss")
)

In [0]:
df_otpw.count()

#### Compare OTPW and Custom Join - 1y

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, sum as _sum

def compare_dataframes(df1: DataFrame, df2: DataFrame, name1: str = "df1", name2: str = "df2"):
    print(f"===== 📊 Comparing {name1} and {name2} =====\n")

    # -----------------------------
    # 1. Row Count
    # -----------------------------
    print("➡ Row Counts:")
    print(f"{name1}: {df1.count()}")
    print(f"{name2}: {df2.count()}\n")

    # -----------------------------
    # 2. Column Names
    # -----------------------------
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)

    print("➡ Columns Present in Both:")
    print(sorted(cols1 & cols2), "\n")

    print("➡ Columns Only in", name1)
    print(sorted(cols1 - cols2), "\n")

    print("➡ Columns Only in", name2)
    print(sorted(cols2 - cols1), "\n")

    # -----------------------------
    # 3. Schema Comparison
    # -----------------------------
    print("➡ Schema Differences:")
    schema1 = {f.name: f.dataType for f in df1.schema.fields}
    schema2 = {f.name: f.dataType for f in df2.schema.fields}

    common = cols1 & cols2

    diffs = {c: (schema1[c], schema2[c]) for c in common if schema1[c] != schema2[c]}

    if diffs:
        for col_name, (t1, t2) in diffs.items():
            print(f" - Column '{col_name}': {name1}={t1}, {name2}={t2}")
    else:
        print("No schema differences.\n")

    # -----------------------------
    # 4. Null Count Comparison
    # -----------------------------
    print("\n➡ Null Counts Per Column:")

    print(f"\n{name1} Null Counts:")
    nulls1 = df1.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df1.columns])
    nulls1.show(truncate=False)

    print(f"\n{name2} Null Counts:")
    nulls2 = df2.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df2.columns])
    nulls2.show(truncate=False)

    # -----------------------------
    # 5. Numeric Summary Comparison (optional)
    # -----------------------------
    numeric_cols1 = [f.name for f in df1.schema.fields if "int" in str(f.dataType) or "double" in str(f.dataType)]
    numeric_cols2 = [f.name for f in df2.schema.fields if "int" in str(f.dataType) or "double" in str(f.dataType)]
    numeric_common = list(set(numeric_cols1) & set(numeric_cols2))

    if numeric_common:
        print("\n➡ Summary Statistics (common numeric columns):")
        print(f"Common numeric columns: {numeric_common}\n")

        print(f"{name1} summary:")
        df1.select(numeric_common).summary().show(truncate=False)

        print(f"{name2} summary:")
        df2.select(numeric_common).summary().show(truncate=False)
    else:
        print("\nNo common numeric columns to summarize.")


In [0]:
compare_dataframes(df_otpw, df_final, "df_otpw", "df_final")
